# Elo ratings based on regular-season games

This notebook implements Elo ratings for NCAA regular-season games using the same formula as FiveThirtyEight's NBA Elo ratings. My resources for this were:

- https://en.wikipedia.org/wiki/Elo_rating_system
- https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/
- https://github.com/fivethirtyeight/nfl-elo-game/blob/master/forecast.py

(The last link above is for 538's NFL Elos (not NBA), but it was useful for a code example of the approach. )

The idea here is to get another feature to be plugged in (alongside seeds, etc.) when predicting tournament games.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from datetime import datetime
from sportsreference.ncaab.boxscore import Boxscores
from sportsipy.ncaab.schedule import Schedule
from sportsreference.ncaab.teams import Teams


C:\Users\socst\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


The following parameter `K` affects how quickly the Elo adjusts to new information. Here I'm just using the value that 538 found most appropriate for the NBA -- I haven't done any analysis around whether this value is also the best in terms of college basketball.

I also use the same home-court advantage as 538: the host team gets an extra 100 points added to their Elo.

In [2]:
K = 20.
HOME_ADVANTAGE = 100.

In [3]:
rs = pd.read_csv(r"C:\Users\socst\Documents\Python Scripts\NCAAB-master\2023 MM\Data\MDataFiles_Stage1\MRegularSeasonCompactResults.csv")
rs.head(3)

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985      20     1228      81     1328      64    N      0
1    1985      25     1106      77     1354      70    H      0
2    1985      25     1112      63     1223      56    H      0

In [4]:
team_ids = set(rs.WTeamID).union(set(rs.LTeamID))
len(team_ids)

377

I'm going to initialise all teams with a rating of 1500. There are two differences here with the 538 approach:

- New entrants (when and where there are any) will start at the average 1500 Elo rather than a lower rating probably more appropriate for a new team.
- There is no reversion to the mean between seasons. Each team's Elo starts exactly where it left off the previous season.  My justification here is that we only care about the end-of-season rating in terms of making predictions on the NCAA tournament, so even if ratings are a little off at first, they have the entire regular season to converge to something more appropriate.

In [5]:
# This dictionary will be used as a lookup for current
# scores while the algorithm is iterating through each game
elo_dict = dict(zip(list(team_ids), [1500] * len(team_ids)))

In [6]:
# Elo updates will be scaled based on the margin of victory
rs['margin'] = rs.WScore - rs.LScore

The three functions below contain the meat of the Elo calculation:

In [7]:
def elo_pred(elo1, elo2):
    return(1. / (10. ** (-(elo1 - elo2) / 400.) + 1.))

def expected_margin(elo_diff):
    return((7.5 + 0.006 * elo_diff))

def elo_update(w_elo, l_elo, margin):
    elo_diff = w_elo - l_elo
    pred = elo_pred(w_elo, l_elo)
    mult = ((margin + 3.) ** 0.8) / expected_margin(elo_diff)
    update = K * mult * (1 - pred)
    return(pred, update)

In [8]:
# I'm going to iterate over the games dataframe using 
# index numbers, so want to check that nothing is out
# of order before I do that.
assert np.all(rs.index.values == np.array(range(rs.shape[0]))), "Index is out of order."

In [9]:
preds = []
w_elo = []
l_elo = []

# Loop over all rows of the games dataframe
for row in rs.itertuples():
    
    # Get key data from current row
    w = row.WTeamID
    l = row.LTeamID
    margin = row.margin
    wloc = row.WLoc
    
    # Does either team get a home-court advantage?
    w_ad, l_ad, = 0., 0.
    if wloc == "H":
        w_ad += HOME_ADVANTAGE
    elif wloc == "A":
        l_ad += HOME_ADVANTAGE
    
    # Get elo updates as a result of the game
    pred, update = elo_update(elo_dict[w] + w_ad,
                              elo_dict[l] + l_ad, 
                              margin)
    elo_dict[w] += update
    elo_dict[l] -= update
    
    # Save prediction and new Elos for each round
    preds.append(pred)
    w_elo.append(elo_dict[w])
    l_elo.append(elo_dict[l])

In [10]:
rs['w_elo'] = w_elo
rs['l_elo'] = l_elo

Let's take a look at the last few games in the games dataframe to check that the Elo ratings look reasonable.

In [11]:
rs.tail(10)

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  margin  \
181515    2023     127     1350      71     1269      38    N      0      33   
181516    2023     127     1386      72     1260      67    N      0       5   
181517    2023     127     1389      70     1193      52    N      0      18   
181518    2023     127     1394      80     1270      63    N      0      17   
181519    2023     127     1436      79     1127      57    H      0      22   
181520    2023     127     1439      67     1323      64    N      0       3   
181521    2023     127     1465      69     1101      62    N      0       7   
181522    2023     127     1467      67     1192      66    H      0       1   
181523    2023     127     1469      80     1372      76    N      1       4   
181524    2023     127     1470      74     1410      70    N      0       4   

              w_elo        l_elo  
181515  1638.237049  1520.833752  
181516  1503.679114  1553.488820  
181517  1391.293255  1358.554557  
181518  1411.096268  1176.191505  
181519  1641.156507  1278.944215  
181520  1832.947142  1664.341932  
181521  1472.145962  1504.825849  
181522  1378.858976  1313.464490  
181523  1419.466510  1547.931368  
181524  1485.794528  1313.724008

Looks OK. How well do they generally predict games? Since all of the Elo predictions calculated above have a true outcome of 1, it's really simple to check what the log loss would be on these 150k games:

In [12]:
np.mean(-np.log(preds))

0.5354759889664574

(This is a pretty rough measure, because this is looking only at regular-season games, which is not really what we're ultimately interested in predicting.)

Final step: for each team, pull out the final Elo rating at the end of each regular season. This is a bit annoying because the team ID could be in either the winner or loser column for their last game of the season..

In [13]:
def final_elo_per_season(df, team_id):
    d = df.copy()
    d = d.loc[(d.WTeamID == team_id) | (d.LTeamID == team_id), :]
    d.sort_values(['Season', 'DayNum'], inplace=True)
    d.drop_duplicates(['Season'], keep='last', inplace=True)
    w_mask = d.WTeamID == team_id
    l_mask = d.LTeamID == team_id
    d['season_elo'] = None
    d.loc[w_mask, 'season_elo'] = d.loc[w_mask, 'w_elo']
    d.loc[l_mask, 'season_elo'] = d.loc[l_mask, 'l_elo']
    out = pd.DataFrame({
        'team_id': team_id,
        'season': d.Season,
        'season_elo': d.season_elo
    })
    return(out)

In [14]:
df_list = [final_elo_per_season(rs, id) for id in team_ids]
season_elos = pd.concat(df_list)

In [15]:
season_elos.sample(10)

team_id  season season_elo
47943      1208    1996     1789.4
78571      1336    2003    1503.77
96785      1152    2007    1340.19
156076     1361    2018    1783.74
170112     1227    2021    1401.51
47951      1390    1996       1783
43719      1264    1995    1663.49
87722      1149    2005    1236.95
43514      1331    1995    1301.34
56179      1435    1998    1724.28

In [16]:
season_elos['season'].max()

2023

In [18]:
season_elos.to_csv(r"C:\Users\socst\Documents\Python Scripts\NCAAB-master\2023 MM\Data\season_elos.csv", index=None)